In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv


In [2]:
df=pd.read_csv('/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv')

**TEXT CLEANING**
1. Removing html tags
1. Removing punctuations from review
1. Removing stray br from br tags, t from /t and n from /n
1. Removing special characters
1. Converting everything to lower case
1. Removing stop words
1. Stemming

In [3]:
# Removing html tags
import re
def remove_html_tags(text):
    pattern = re.compile('<.*?>')
    return pattern.sub(r'',text)
df['review']=df['review'].apply(remove_html_tags)
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. The filming tec...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [4]:
df['sentiment'].replace({'positive':1,'negative':0},inplace=True)

In [5]:
# to remove punctuations
import string
regular_punct = list(string.punctuation)
def remove_punctuation(text):
    for punc in regular_punct:
        if punc in text:
            text = text.replace(punc, ' ')
    return text.strip()
df['review']=df['review'].apply(remove_punctuation)

In [6]:
df.head()
df.iloc[9].review

'If you like original gut wrenching laughter you will like this movie  If you are young or old then you will love this movie  hell even my mom liked it Great Camp'

In [7]:
# Removing stray br from br tags, t from /t and n from /n
def brRemove(text):
    x=''
    ans=''
    for i in text:
        if i==' ':
            if x!='br' and x!='t'and x!='n':
                ans=ans+' '+x
            x=''
        else:
            x=x+i
    return ans
df['review']=df['review'].apply(brRemove)
df.head

<bound method NDFrame.head of                                                   review  sentiment
0       One of the other reviewers has mentioned that...          1
1       A wonderful little production  The filming te...          1
2       I thought this was a wonderful way to spend t...          1
3       Basically there s a family where a little boy...          0
4       Petter Mattei s  Love in the Time of Money  i...          1
...                                                  ...        ...
49995   I thought this movie did a down right good jo...          1
49996   Bad plot  bad dialogue  bad acting  idiotic d...          0
49997   I am a Catholic taught in parochial elementar...          0
49998   I m going to have to disagree with the previo...          0
49999   No one expects the Star Trek movies to be hig...          0

[50000 rows x 2 columns]>

In [8]:
# Convert to lower case
def convert_lower(text):
    return text.lower()
df['review']=df['review'].apply(convert_lower)

In [9]:
# Removing special characters
def remove_special(text):
    x=''
    
    for i in text:
        if i.isalnum():
            x=x+i
        else:
            x=x+' '
    return x

In [10]:
df['review']=df['review'].apply(remove_special)

In [11]:
# Removing stopwords
import nltk
from nltk.corpus import stopwords
def remove_stopwords(text):
    
    x=[]
    for i in text.split():
        if i not in stopwords.words('english'):
            x.append(i)
    y=x[:]
    x.clear()
    return y


In [12]:
df['review']=df['review'].apply(remove_stopwords)
df.head()

,review,sentiment
0,"[one, reviewers, mentioned, watching, 1, oz, e...",1
1,"[wonderful, little, production, filming, techn...",1
2,"[thought, wonderful, way, spend, time, hot, su...",1
3,"[basically, family, little, boy, jake, thinks,...",0
4,"[petter, mattei, love, time, money, visually, ...",1


In [13]:
# Stemming
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
y=[]
def stem(text):
    for i in text:
        y.append(ps.stem(i))
    z=y[:]
    y.clear()
    return z


In [14]:
df['review']=df['review'].apply(stem)

# joining it back
def join_back(list_input):
    return " ".join(list_input)

In [15]:
df['review']=df['review'].apply(join_back)


In [16]:
# Bag of words
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=1000,binary=True)
X = cv.fit_transform(df['review']).toarray() 
y = df.iloc[:,-1].values 

In [17]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

In [18]:
from sklearn.naive_bayes import BernoulliNB,MultinomialNB,GaussianNB
from sklearn.metrics import accuracy_score
model1 = MultinomialNB()
model2 = BernoulliNB()
model3 = GaussianNB()

model1.fit(X_train,y_train)
model2.fit(X_train,y_train)
model3.fit(X_train,y_train)

y_pred1 = model1.predict(X_test)
y_pred2 = model2.predict(X_test)
y_pred3 = model3.predict(X_test)

In [19]:
print("Multinomial Accuracy",accuracy_score(y_test,y_pred1))
print("Bernoulli Accuracy",accuracy_score(y_test,y_pred2))
print("Gaussian Accuracy",accuracy_score(y_test,y_pred3))

Multinomial Accuracy 0.8402
Bernoulli Accuracy 0.841
Gaussian Accuracy 0.8204
